In [26]:
from re import S
import pandas as pd
import nltk.corpus
nltk.download("stopwords")
from nltk.corpus import stopwords

def preprocessing(fname):
  df = pd.read_csv(fname, sep="\t")

  df.columns = ["ID", "Label", "Statement", "Subject", "Speaker", "Job Title", "State Info", 
          "Party Affiliation", "Credit History", "False Counts", "Half True Counts", "Mostly True", "Pants on Fire",
          "Context"]
  df.drop(["ID", "Speaker", "Job Title", "State Info", "Context", "Party Affiliation",
          "Credit History", "Subject", "False Counts", "Half True Counts", "Mostly True", "Pants on Fire"], inplace=True, axis=1)

  df["Statement_without_stopwords"] = df["Statement"].apply(lambda x: ' '.join([_ for _ in x.split() if _ not in (stopwords.words("english"))]))

  df["Label"] = df["Label"].map({"true": "Real",
                          "mostly-true": "Real", 
                          "half-true": "Fake",
                          "barely-true": "Fake" ,
                          "false": "Fake",
                          "pants-fire": "Fake"})
  df["Label"].replace({})
  return df["Statement_without_stopwords"], df["Label"]

X_train, y_train = preprocessing("train.tsv")
X_test, y_test = preprocessing("test.tsv")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/NiravShah/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


tok = Tokenizer()
tok.fit_on_texts(X_train)
v_size = len(tok.word_index) + 1
padded_reviews = pad_sequences(tok.texts_to_sequences(X_train), maxlen=150, padding='post')

In [34]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, SpatialDropout1D
from tensorflow.keras.optimizers import RMSprop
import tensorflow.keras.backend as K

model = Sequential()
model.add(Embedding(v_size, 32, input_length=150))
# While passing output to next LSTM layer set return_sequence to Ture.
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

NotImplementedError: Cannot convert a symbolic Tensor (lstm_19/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported